# Documentation Neuronal Network



In [44]:

# We will use this cell to import all the packages you will need in the following - think of it as turning on all your systems
# in your cockpit

# This makes sure that if you change code in your external scripts, they will be updated

from IPython.display import display, clear_output
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow import keras
from keras.api._v2.keras import layers
import tensorflow as tf
import os
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import datetime
import umap

cwd = os.getcwd ()

# now go ahead and Run the cell. This might take a while...
# while the cell is running, you will see ln[*] next to it. Once it finished, you will see the number of execution
# In case you want to interrupt the run of a cell, press Ctrl + C (on your german keyboard, this is Strg + C) 



### Defining the model and data parameters

Now we define our parameters to use them later


In [45]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
keep_index = [39306, 45926, 35779, 9059, 3639, 29049, 15459, 26151, 56284, 1733]

## Exploring the MNIST dataset

Before any data science or machine learning project, it is essential to get to know your data. This will enable you to detect issues, noise, pitfalls and understand what your model will learn at the end of the day. With the MNIST dataset, you will be working with a beautiful, cleaned, easy to understand, low-memory, large-scale dataset. 
Full disclaimer, MNIST is excellent for learning and research purposes, yet this is not what you can expect in real-life. 

This initially becomes obvious as MNIST is so commonly used that the Keras packages got our back with loading the data in one line. We are loading pairs of samples and ground truth annotations for both the training set (for training our model) and the test set (for testing our model). 



In [46]:
# Loading the MNIST dataset in one line
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Printing the shape
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

x_train: (60000, 28, 28)
y_train: (60000,)
x_test: (10000, 28, 28)
y_test: (10000,)


## Defining our function to calculate the mse of two pictures

With this function we want to calculate the mse of two labels to get the most similar ones


In [47]:
def calculate_mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err

x_train = x_train[0:10000]
y_train = y_train[0:10000]


In [48]:
def search_for_most_similar_input(list, index):
  store_mse = 1.0
  remember_index = 1
  for i in range (index, len(list)):
    if i != index: 
      actual_mse = calculate_mse(list[index], list[i])
      if actual_mse < store_mse:
        store_mse = actual_mse
        remember_index = i
  return remember_index
for calc in range (0,8):
  for i in range (0,10):
    filter_for_number = np.where(y_train == i)[0]
    for y in range (len(x_train[filter_for_number])//2):
      index = search_for_most_similar_input(x_train[filter_for_number], y)
      x_train = np.delete(x_train, filter_for_number[index], 0)
      y_train = np.delete(y_train,filter_for_number[index])
      filter_for_number = np.where(y_train == i )[0]

print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

x_train: (43, 28, 28)
y_train: (43,)
x_test: (10000, 28, 28)
y_test: (10000,)


In [ ]:
keep_numbers=[]
mse_numbers = []
def searchBestImage():
  for number in range (0,10):
    mse_number = 0.0
    filter_for_number = np.where(y_train == number)[0]
    for number1 in range (len(filter_for_number)):
      for number2 in range (len(filter_for_number)):
        mse_number += calculate_mse(x_train[filter_for_number[number1]],x_train[filter_for_number[number2]])
      mse_number = mse_number / len(filter_for_number)
      mse_numbers.append(mse_number)
    min = np.argmin(mse_numbers)
    keep_index.append(filter_for_number[min])
    mse_numbers.clear()
    filter_for_number = np.where(y_train == number )[0]


In [19]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [20]:

#searchBestImage()

#x_train = x_train[keep_index]
#y_train = y_train[keep_index]

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape: ", x_train.shape)
print("y_train shape: ", y_train.shape)
print("x_test shape", x_test.shape)
print("y_test shape", y_test.shape)


x_train shape:  (43, 28, 28, 1)
y_train shape:  (43,)
x_test shape (10000, 28, 28, 1)
y_test shape (10000,)


In [39]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
marvin = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(256, activation='sigmoid',kernel_initializer='glorot_uniform'),
  tf.keras.layers.Dense(128, activation='sigmoid',kernel_initializer='glorot_uniform'),
  tf.keras.layers.Dropout(0.1),#
  tf.keras.layers.Dense(9, activation='softmax',kernel_initializer='glorot_uniform', use_bias=True)
])


In [43]:
batch_size = 8
epochs = 150

#print("Number of input:",lda.n_components_)
keep_index = [39306, 45926, 35779, 9059, 3639, 15459, 26151, 56284, 1733]

x_train=x_train[keep_index]
y_train=y_train[keep_index]
print("y_train:",y_train)
print('x_red:', x_train.shape)

marvin.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.legacy.Adam(learning_rate=0.01), metrics=["accuracy"])

marvin.summary()

#%%
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#%%
early_stopping = tf.keras.callbacks.EarlyStopping (monitor='val_accuracy',patience=15,mode="max") 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau (monitor ='val_accuracy', factor =0.5 , min_delta=0.00005 , 
                                                  patience=6, min_lr=0.001, mode="max")
model_checkpoint = tf.keras.callbacks.ModelCheckpoint (filepath=os.path.join(cwd,'models/mnist_10_label.h5'),
                                                       monitor='val_accuracy',save_best_only=True, mode="max")

marvin.fit(
    x_train,
    y_train,
    epochs= 200,
    batch_size= 32,
    validation_data=(x_test, y_test),
    callbacks=[model_checkpoint,reduce_lr]
)

marvin_reloaded = tf.keras.models.load_model('models/mnist_10_label.h5')
loss_and_metrics = marvin_reloaded.evaluate(x_test, y_test, verbose=2)

print("Test Loss", loss_and_metrics[0])
print("Test Accuracy", loss_and_metrics[1])

y_train: [0 1 2 3 4 6 7 8 9]
x_red: (9, 28, 28)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               200960    
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 9)                 1161      
                                                                 
Total params: 235017 (918.04 KB)
Trainable params: 235017 (918.04 KB)
Non-trainable params: 0 (0.00 Byte)
________________________________

ValueError: in user code:

    File "/Users/nikolasgross/Desktop/venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/nikolasgross/Desktop/venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/nikolasgross/Desktop/venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/nikolasgross/Desktop/venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/nikolasgross/Desktop/venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/Users/nikolasgross/Desktop/venv/lib/python3.11/si

In [ ]:
# Let Marvin predict on the test set, so we have some data to evaluate his performance.
predictions = marvin_reloaded.predict([x_test])

# Remember that the prediction of Marvin is a probability distribution over all ten-digit classes
# We want him to assign the digit class with the highest probability to the sample.
predictions = np.argmax(predictions, axis=1)
#pd.DataFrame(predictions)
